# Interactive Visualisation and Dashboards

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pypsa atlite pandas geopandas xarray matplotlib hvplot geoviews plotly highspy
%env HV_DOC_HTML=true
```
:::

In [ ]:
import pypsa
import atlite
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import holoviews as hv

plt.style.use("bmh")

In [ ]:
from urllib.request import urlretrieve
from os.path import basename

urls = [
    "https://tubcloud.tu-berlin.de/s/2oogpgBfM5n4ssZ/download/PORTUGAL-2013-01-era5.nc",
]
for url in urls:
    urlretrieve(url, basename(url))

## Load Example Data

First, let's load a few example datasets you know from previous tutorials.

A PyPSA network:

In [ ]:
n = pypsa.Network(
    "https://tubcloud.tu-berlin.de/s/kpWaraGc9LeaxLK/download/network-cem.nc"
)

In [ ]:
n.optimize(solver_name="highs");

Wind, solar and demand time series:

In [ ]:
url = (
    "https://tubcloud.tu-berlin.de/s/nwCrNLrtL6LAN3W/download/time-series-lecture-2.csv"
)
ts = pd.read_csv(url, index_col=0, parse_dates=True)

Power plants in Europe

In [ ]:
url = (
    "https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv"
)
ppl = pd.read_csv(url, index_col=0)
geometry = gpd.points_from_xy(ppl["lon"], ppl["lat"])
ppl = gpd.GeoDataFrame(ppl, geometry=geometry, crs=4326)

NUTS2 regions:

In [ ]:
url = "https://tubcloud.tu-berlin.de/s/RHZJrN8Dnfn26nr/download/NUTS_RG_10M_2021_4326.geojson"
nuts = gpd.read_file(url).set_index("id").query("LEVL_CODE == 2")

An `atlite` cutout:

In [ ]:
cutout = atlite.Cutout("PORTUGAL-2013-01-era5.nc")

## Limitations of Static Plotting with Matplotlib

You will agree that using `matplotlib` for static plotting is great for reports, but that it's lacking some features for interactive visualisation.

In [ ]:
ts["onwind [pu]"].plot(figsize=(10, 2))

There are many Python-based interactive plotting libraries out there, and it can be [confusing to keep an overview](https://medium.com/mlearning-ai/top-python-libraries-for-data-visualization-static-and-interactive-visualization-e5f1bc72de41). This tutorial introduces you to two of them:

- [hvPlot](https://hvplot.holoviz.org/index.html), which is a high-level API mostly for [bokeh](https://docs.bokeh.org/en/latest/) plots that integrates nicely with `pandas`.
- [plotly.express](https://plotly.com/python/plotly-express/), which is a high-level API for [plotly](https://plotly.com) plots.

These two tools allow you to produce shiny interactive figures with minimal code, however, at the expense of fewer customisation options.

## hvPlot

<img src="https://hvplot.holoviz.org/assets/diagram.svg" width="900px" /> 

`.hvplot()` is a powerful and interactive Pandas-like `.plot()` API. You just replace `.plot()` with `.hvplot()` and you get an interactive figure. Simple as that.

Documentation can be found here: https://hvplot.holoviz.org/index.html

To use it, we have to import `hvplot.pandas`, which makes the `.hvplot` accessor available on Pandas DataFrame and Series objects, which means that after that `df.hvplot` becomes a valid statement while before that it would raise an error.

In [ ]:
import hvplot.pandas

Let's try it by plotting onshore wind time series for the year...

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
ts["onwind [pu]"].hvplot(height=200)

... or the load time series for February

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
ts.loc["2015-02", "load [GW]"].hvplot(height=200)

We can also plot geographic data with **hvPlot**, for instance, the locations of all hard coal power plants in Europe.

The `geo=True` declares that the data will be plotted in a geographic coordinate system.
Once **hvPlot** knows that your data is in geo-coordinates, you can use the `tiles` keyword argument to overlay a the plot on top of map tiles.

:::{note}
For a list of available tiles, look [here](https://holoviews.org/reference/elements/bokeh/Tiles.html).
:::

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
ppl.query("Fueltype == 'Hard Coal'").hvplot(
    geo=True, tiles=True, frame_height=600, frame_width=600
)

Like in `geopandas`, we can tell **hvPlot** to plot the point sizes and colors according to columns of the `pandas.DataFrame`. We can also change the opacity with `alpha` and the colormap with `cmap`. 

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
plot = ppl.query("Fueltype == 'Hard Coal'").hvplot(
    geo=True,
    tiles="CartoLight",
    frame_height=600,
    c="DateIn",
    cmap="viridis",
    s="Capacity",
    alpha=0.6,
)
plot

There are a few more options of the graph we can tweak in the `opts()` section, like which tools should be activated by default.

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
plot = plot.opts(xaxis=None, yaxis=None, active_tools=["pan", "wheel_zoom"])
plot

All this does not only work with points but also shapes. We can also pick the columns that should be shown when hovering on a shape using `hover_cols`.

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
nuts.hvplot(
    geo=True,
    tiles="OSM",
    hover_cols=["NUTS_NAME", "NUTS_ID"],
    c="CNTR_CODE",
    frame_height=500,
    alpha=0.2,
).opts(xaxis=None, yaxis=None, active_tools=["pan", "wheel_zoom"])

We can also use **hvPlot** for `xarray` datasets (e.g. `atlite` cutouts).

For that, we need to import the corresponding `xarray` accessors.

In [ ]:
import hvplot.xarray

So let's try it by plotting the wind speeds in Portugal as provided by ERA5. The nice thing you will notice is that it will automatically open a panel for dimensions that we did not select explicitly. In this case we can easily sweep across the time dimension. Notice also the customisation options we use here.

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
cutout.data.hvplot.quadmesh(
    "x",
    "y",
    "wnd100m",
    frame_height=500,
    cmap="Blues",
    geo=True,
    tiles="CartoLight",
    alpha=0.8,
    padding=0.5,
    clim=(0, 10),
)

We can also plot the time series of solar generation in Germany on a heatmap:

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
ts.hvplot.heatmap(
    x="index.hour", y="index.month", C="solar [pu]", cmap="blues"
).aggregate(function="mean")

**hvPlot** also offers stacked area charts that come in handy for plotting the power dispatch of a solved PyPSA network:

In [ ]:
dispatch = (
    pd.concat([n.generators_t.p, n.storage_units_t.p], axis=1).loc["2015-02"].div(1e3)
)

In [ ]:
# for Google Colab add:
# hv.extension('bokeh')
dispatch.where(dispatch > 0, 0).hvplot.area(
    stacked=True,
    line_width=0,
    width=1300,
    height=350,
    hover=False,
    color=[n.carriers.at[c, "color"] for c in dispatch.columns],
    ylabel="electricity supply [GW]",
    ylim=(0, 180),
)

## Plotly Express

> The `plotly.express` module (usually imported as px) contains functions that can create entire figures at once. Plotly Express is a built-in part of the `plotly` library, and is the recommended starting point for creating most common figures. Every Plotly Express function uses graph objects internally and returns a plotly.graph_objects.Figure instance. Throughout the plotly documentation, you will find the Plotly Express way of building figures at the top of any applicable page, followed by a section on how to use graph objects to build similar figures. Any figure created in a single function call with Plotly Express could be created using graph objects alone, but with between 5 and 100 times more code.

Documentation is available here: https://plotly.com/python/plotly-express/

In [ ]:
import plotly.io as pio
import plotly.express as px
import plotly.offline as py

:::{note}
We need to import `plotly.io` and `plotly.offline`, so that the interactive plots are also visible on the course's static website.
:::

Let's reproduce the plots we previously created with **hvPlot**. Onshore wind capacity factor time series:

In [ ]:
px.line(ts["onwind [pu]"])

Load time series in February:

In [ ]:
px.line(ts.loc["2015-02", "load [GW]"])

Hard coal power plants in Europe:

In [ ]:
df = ppl.query("Fueltype == 'Hard Coal'")

In [ ]:
px.scatter_mapbox(
    df, lat="lat", lon="lon", mapbox_style="carto-positron", zoom=2, height=600
)

In [ ]:
px.scatter_mapbox(
    df,
    lat="lat",
    lon="lon",
    mapbox_style="carto-positron",
    color="DateIn",
    size="Capacity",
    zoom=2,
    height=600,
)

In [ ]:
px.choropleth_mapbox(
    nuts,
    geojson=nuts.geometry,
    locations=nuts.index,
    mapbox_style="carto-positron",
    zoom=2,
    height=600,
    color="CNTR_CODE",
    center={"lat": 48, "lon": 12},
)

The integration with `xarray` datasets is not as nice as in **hvPlot**.

In [ ]:
px.imshow(cutout.data.wnd100m[:, :, 0].T)

But in `plotly`, hovering information on the area chart works much better.

In [ ]:
dispatch = (
    pd.concat([n.generators_t.p, n.storage_units_t.p], axis=1).loc["2015-02"].div(1e3)
)

In [ ]:
df = (
    dispatch.where(dispatch > 0, 0)
    .stack()
    .reset_index()
    .rename(columns={"level_1": "technology", 0: "GW"})
)

In [ ]:
fig = px.area(df, x="snapshot", color="technology", y="GW", line_group="technology")
fig.update_traces(line=dict(width=0))
fig

## Interactive Dashboards

There are many different options for building interactive dashboards. Some are brand new, some have been around for a few years.

<img src="https://global-uploads.webflow.com/5d3ec351b1eba4332d213004/5f99e10dafbd69a99c875340_C8_qX8dvzv60T4LVZ9GftX-ZH-VJzq3sjUroWWH5XSWw8RFHnCCPPrC6jB3EFVuQdwiqhoEMQKFV-dFz7t6fqaRpSZGvBKI0i1Utj38_j9a54GXMuzi1BiepdIMjOK4ATVdF2131.png" width="900px" /> 

Each of them has different characteristics, for instance in terms of customisation options and ease of use.

<img src="https://global-uploads.webflow.com/5d3ec351b1eba4332d213004/5fa3b0d4a2043bcf84d49134_z87mnMfsPGOF7L3sGULQBusJnJTWGZHWtoizufuDR1q1A6JggFWO9IYHXf8wFyqgKhuG6hEGOPM4Acb-qmNXxwCFW95DPX9r7Syewkejb7itbmm8I_os2XI8bightYGJq7Gg-FXo.png" width="900px" /> 

If you want to read a detailed comparison, the best one I found is this one:

https://www.datarevenue.com/en-blog/data-dashboarding-streamlit-vs-dash-vs-shiny-vs-voila


> **Just tell me which one to use**
>
> As always, “it depends” – but if you’re looking for a quick answer, you should probably use:
>
>    - Dash if you already use Python for your analytics and you want to build production-ready data dashboards for a larger company.
>    - **Streamlit if you already use Python for your analytics and you want to get a prototype of your dashboard up and running as quickly as possible.**
>    - Shiny if you already use R for your analytics and you want to make the results more accessible to non-technical teams.
>    - Jupyter if your team is very technical and doesn’t mind installing and running developer tools to view analytics.
>    - Voila if you already have Jupyter Notebooks and you want to make them accessible to non-technical teams.
>    - Flask if you want to build your own solution from the ground up.
>    - Panel if you already have Jupyter Notebooks, and Voila is not flexible enough for your needs.

In this tutorial, we look at `streamlit` because it is the easiest to get to results quickly. However, compared to other dashboarding libraries, it has more limited configuration options.

Documentation for this package can be found here: https://docs.streamlit.io/

Streamlit can be installed, for example, with `conda`, `mamba` or `pip`:

```sh
conda install -c conda-forge streamlit'>=1.18'
```

or

```sh
pip install streamlit
```

:::{note}
This tutorial requires `streamlit>=1.18`.
:::

This tutorial is stored on Github with instructions how to install, run and deploy it:

https://github.com/fneum/streamlit-tutorial

You can see a live demo of the final product here:

https://ppm-dash.streamlit.app/
